In [ ]:

# Mendelian Randomization using TwoSampleMR
***

In [3]:
from datetime import date
today = date.today()
date= today.strftime("%B %d, %Y")
import getpass
import os
cwd = os.getcwd()

In [4]:
print("Date: ", date)
print("")
print("Analysed by: ", getpass.getuser())
print("")
print("Current working directory: {0}".format(cwd))
print("")

Date:  March 20, 2023

Analysed by:  saezs

Current working directory: /vf/users/ALS_50k/SaraSaez_ALS/2022_01_06.ALS.DrugsRepositioning





# Workflow

***


<b>  (1) Prepare a list of the traits related to hypertension to be analyzed </b>  (hypertension.2sampleMR.txt) 
   
       
<br> (2) Perform Mendelian Randomization using TwoSampleMR </b>


In [11]:
%%bash
cd /data/ALS_50k/SaraSaez_ALS/2022_01_06.ALS.DrugsRepositioning

echo "id
finn-b-I9_HYPTENSESS_EXNONE
finn-b-I9_HYPTENSESS
finn-b-I9_HYPTENS_EXNONE
finn-b-I9_HYPTENSEC_EXNONE
finn-b-I9_HYPTENSPREG
finn-b-O15_GESTAT_OEDEM_PREINUR
finn-b-G6_BENINTRAHYP
finn-b-I9_HYPTENSEC
finn-b-O15_PREECLAMPS_CHRONIC_HYPERT
finn-b-O15_GESTAT_HYPERT
finn-b-I9_HYPTENSPUL_EXNONE
finn-b-I9_HYPTENSPUL
ukb-e-401_AFR
ukb-e-401_CSA
ukb-e-401_EAS
ukb-e-401_MID
ebi-a-GCST008036
ukb-b-16587
ukb-b-13535
ukb-b-12493
ukb-b-7582
ukb-b-14057
ukb-d-I9_HYPTENS
ukb-a-531
ukb-a-61
ukb-b-14177
ukb-b-18167
ukb-b-19456
ukb-b-8746
ukb-a-437
ukb-a-215
ukb-a-207
ukb-a-222
ukb-b-1061" > hypertension.2sampleMR.txt

# Analysis
***

In [ ]:
%%bash
cd /data/ALS_50k/SaraSaez_ALS/2022_01_06.ALS.DrugsRepositioning
mkdir Results
module load R/4.1.0
R --vanilla --no-save
#load packages
library(data.table)
library(TwoSampleMR)
library(MRInstruments)
library(RadialMR) 
library(ieugwasr)
library(dplyr)
library(tidyverse)
sessionInfo()
#The standard error is actually estimated using bootstrapping which is a stochastic procedure.
#this will give a different result each time and also a different p-value as a consequence. 
#Could ensure reproducibility by setting the same seed at the beginning of the session
set.seed(1214) 

VR <- fread("/data/ALS_50k/SaraSaez_ALS/2022_01_06.ALS.DrugsRepositioning/GWAS/ALS_sumstats_EUR_only.txt", header = T)
VR$SNP = VR$rsid
VR <- dplyr::select(VR, SNP, effect_allele, other_allele, effect_allele_frequency, beta, standard_error, p_value, N_effective)

Out_data = format_data(VR,type="outcome", beta_col = "beta", se_col = "standard_error", eaf_col = "effect_allele_frequency", pval_col = "p_value")

 

token <-  "ya29.A0ARrdaM9H5otEAHugyicLDAsRSkskkRjYgOC0tWvq_RynTztv1u3cDHiXMCq58iZ4EWKGpV8AIT4zKRQ0J0IPyfF7P77JGW38YD7A5ysweNZrys1wwYhBr-poTOBXIZWSu3d1-eWATMxmbZ0EUD1Ey0RAfVbL"

possibleInstruments <- available_outcomes(access_token = token)
listOfGwasIds <- read.table("hypertension.2sampleMR.txt", header = T)
for(i in 20:20)
{
  instrumentId <- as.character(listOfGwasIds$id[i])
   tag <- paste("INSTRUMENT IS ",instrumentId," AT i = ",i, sep = "")
 print(tag)
flagged <- "nope"
Exp_data <- extract_instruments(outcomes=instrumentId, p1 = 5e-08, clump = TRUE, p2 = 5e-08,
                                  r2 = 0.001, kb = 10000, access_token = token,
                                 force_server = TRUE)
skip <- ifelse(length(Exp_data$beta.exposure) < 1, 1, 0)
if(skip == 0)
{
dat <- harmonise_data(exposure_dat=Exp_data, outcome_dat=Out_data, action=2)
res <-mr(dat)
 print(res)
het <- mr_heterogeneity(dat)
 print(het)
ple <- mr_pleiotropy_test(dat)
 print(ple)
write.table(res, file = paste(instrumentId,".HY.1res.txt",sep = ""), quote = F, sep = ",")
write.table(het, file = paste(instrumentId,".HY.2het.txt",sep = ""), quote = F, sep = ",")
write.table(ple, file = paste(instrumentId,".HY.3ple.txt",sep = ""), quote = F, sep = ",")
res_single <- mr_singlesnp(dat)
mr_report(dat)
write.table(res_single, file = paste(instrumentId,"HY.4res.txt",sep = ""), quote = F, sep = ",")
 }
  else
  {
    print("FAIL")
  }
}

